Welcome to Lecture 3.2. In this exercise, you will train several ensemble models such as Random Forest, Gradient Boosting and XGBoost. Then you need to combine these models to implement stacking.

In [ ]:
# import numpy and matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = [10, 6]

# skip all warnings
import warnings
warnings.filterwarnings('ignore')

# Prepare MNIST Data

You will be using [MNIST database](https://en.wikipedia.org/wiki/MNIST_database) as our training dataset. First, let's load mnist data from `sklearn.datasets`. Then, you will split 70000 data instances into training set (50000), validation set (10000) and test set (10000). You should use training set to train model and validation set to evaluate the model. Then you compare the various models to find the best one. Once you have found it, try it on test set. Note that we will not include much hyperparameters tuning in this notebook. We will do it in next Lecture. 


* Use `sklearn.model_selection.train_test_split()`.
* You can split twice. For the first split, you get `train_val` and `test`. Then you split `train_val` to get `train` and `test`.
* Both of the `random_state` should be set to 42.
* Name them **X_train, y_train, X_val, y_val, X_test, y_test**.

In [ ]:
# fetch data from sklearn
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split

mnist = fetch_mldata('MNIST original')
X, y = mnist['data'], mnist['target']

# first split
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=10000, random_state=42)

# second split
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=10000, random_state=42)

# Random Forest, Gradient Boosting

In this section, we are going to initialize two classification models. We will train models in section 4. 
* [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) 
* [Gradient Boosting](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)

Random Forest and Gradient Boosting are both in `sklearn.ensemble`. The hyperparameters of these two models has already been assigned for you. Again, you might tune them by hand or using [GridSearchCV()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html), but this is not our objective today.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest hyperparameters, DO NOT MODIFY
n_estimators = 50
max_depth = None
min_samples_split = 2
max_leaf_nodes = None
random_state=42

# initialize a Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=n_estimators, 
                                max_leaf_nodes=max_leaf_nodes, 
                                max_depth=max_depth, 
                                min_samples_split=min_samples_split,
                                random_state=random_state,
                                verbose=True,
                                n_jobs=-1)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Gradient Boosting hyperparameters, DO NOT MODIFY
n_estimators = 30 # you might change to a higher one
learning_rate = 0.3
min_samples_split = 0.01
min_samples_leaf = 10
max_depth = 2
subsample = 0.8
random_state=42

# initialize a Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(n_estimators=n_estimators, 
                                    learning_rate=learning_rate,
                                    min_samples_split=min_samples_split,
                                    min_samples_leaf=min_samples_leaf, 
                                    max_depth=max_depth, 
                                    subsample=subsample,
                                    random_state=random_state,
                                    verbose=True)

# XGBoost

[XGBoost](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/) (e$X$treme $G$radient $Boost$ing) is an advanced implementation of gradient boosting algorithm. It has become the ultimate weapon of many data scientist. I highly recommend to use XGboost, if things don’t go your way in predictive modeling. Building a XGBoost model is very easy, but improving it is difficult. Thus hyperparameter tuning is must for XGBoost. 

XGBoost has many advantages. Here I list four important advantages, which might be useful for interview.
1. **Regularization**. Standard GBM implementation has no regularization, which is easy to overfitting. Regularization helps XGBoost to reduce overfitting.
2. **Parallel Processing**. XGBoost is faster compared to GBM. To explore details, check [this](http://zhanpengfang.github.io/418home.html).
3. **Handling Missing Values**. XGBoost has an in-built routine to handle missing values.
4. **Built-in Cross-Validation**. XGBoost allows user to run a cross-validation at each iteration of the boosting process and thus it is easy to get the exact optimum number of boosting iterations in a single run.

****

In Python, there is a Scikit-Learn API for XGBoost, making it easier to implement. As before, I have set up hyperparameters for you. This can be the start points. But in reality or in Kaggle Competition, you should tune hyperparameters very carefully for XGBoost. Now let's initialize XGBoost and train it in next section.

* [XGBoost](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn)


**NOTE:** You should install xgboost package before using it. Run `pip install xgboost` in your terminal, or `! pip install xgboost` in this jupyter notebook.

In [ ]:
import xgboost

# XGBoost hyperparameters, DO NOT MODIFY
n_estimators = 30 # you might change to a higher one
learning_rate = 0.3
max_depth = 2
min_child_weight = 5
gamma = 0.1
subsample = 0.8
random_state = 42

# initialize a XGBoost Classifier
xgb_clf = xgboost.XGBClassifier(n_estimators=n_estimators, 
                                learning_rate=learning_rate,
                                max_depth=max_depth, 
                                min_child_weight=min_child_weight,
                                gamma=gamma,
                                subsample=subsample,
                                random_state=random_state,
                                verbosity=2,
                                n_jobs=12)

# Stacking Ensemble

The stacking idea is very simple. Instead of using trivial functions (such as hard voting) to aggregate the results of all estimators, we can train a model (meta-estimator) to perform this aggregation. As shown in the picture below,

* First, the individual classification models are trained based on the training set. 
* Then we predict on validation set `X_val`, named `X_val_predictions`.
* Third, a meta-classifier is fitted based on `X_val_predictions` and `y_val`. 


Reference: [StackingClassifier](http://rasbt.github.io/mlxtend/user_guide/classifier/StackingClassifier/)
<img src="./test/stacking.png" width="400">

## Train Individual Models

First, let's train three individual models and print out the accuracy score on validation set. It might take a long time to run Gradient Boosting and XGBoost. Thus, you should record accuracy scores as well as running time for each model. Here are some results of Gradient Boosting Classifier for your reference. Generally, there is a positive correlation between running time and number of estimators. Also the accuracy score increases significantly when number of estimators is larger. You may choose `n_estimator` up to 500 for Gradient Boosting or XGBoost, if you have a strong machine or you would like to spend a long time training model. But here I suggest you choose 30. After you go over all the cells in this notebook, you might choose a higher one. 

* n_estimators = 10, run ~5 mins, score ~0.85
* n_estimators = 30, run ~10 mins, score ~0.91
* n_estimators = 100, run ~50 mins, score ~0.94
* n_estimators = 200, run ~2 hours, score ~0.96
* n_estimators = 500, run ~4 hours, score ~0.97

**NOTE:** In reality we use cross validation to tune hyperparameters, which will definitely spend tons of time for Gradient Boosting. One efficient way to solve this time-consuming problem is to down-sampling your training data. We will soon meet this problem in Kaggle Competition.

In [ ]:
import time

estimators = [rf_clf, gb_clf, xgb_clf]
scores = []
times = []
for estimator in estimators:
    # record start time
    start = time.time()
    # fit data for each classifier
    estimator.fit(X_train, y_train)
    # save spend time
    times.append(time.time()-start)
    # save accuracy score
    scores.append(estimator.score(X_val, y_val))

print ("Accuracy scores for Random Forest, Gradient Boosting and XGBoost are:")
print (scores)
print ("Running time for Random Forest, Gradient Boosting and XGBoost are:")
print (times)

## Stacking

Next, let's stack three models to get a new one. First, we get the predictions on validation set. Each model will have a list of prediction results. We vertically append those lists of results as `X_val_predictions`. To be specific, we initialize an empty array, which has the shape of `(number of samples in X_val, number of estimators)`. Then we make prediction one by one and insert the results into the right position. 

For the meta-classifier, we use Random Forest model. To compare with other models, we should take a look at the oob score of this Random Forest. 

In [ ]:
# initialization
X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)

# insert results to X_val_predictions
for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)

# initialize meta classifier
meta_clf = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=42)

# fit X_val_predictions and y_val
meta_clf.fit(X_val_predictions, y_val)

# compute oob score of meta classifier
oob_score = meta_clf.oob_score_

We might see that stacking is not as good as using single Random Forest Classifier. The reason is that we only use one pair of train/validate in stacking. Actually, we should use Stacking Cross Validation in order to get a more accurate result. Here we only introduce what is stacking and how to implement a simple stacking model. We will go into details in Lecture 3.3.

## Performance On Test Set

At last, let's compare the performance of all models on test set. For stacking, we need to get `X_test_predictions` using the same way as getting `X_val_predictions`.

In [ ]:
from sklearn.metrics import accuracy_score

# initialize X_test_predictions
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

# get X_test_predictions
for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)

# record accuracy scores of four models
test_scores = []

# compute scores for individual classifiers
for estimator in estimators:
    # save accuracy score
    test_scores.append(estimator.score(X_test, y_test))

# predict on X_test_predictions using meta_clf 
y_pred_meta = meta_clf.predict(X_test_predictions)

# append accuracy score of meta classifier
test_scores.append(accuracy_score(y_test, y_pred_meta))